In [1]:
print("hello")

hello


In [2]:
pip install clawpack

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from clawpack import riemann, pyclaw
import sys
import os
from google.colab import drive

# Mount Google Drive if you haven't already
drive.mount('/content/drive')

# Assuming your project is in Google Drive
# Adjust this path to point to your project's root directory
project_path = '/content/drive/MyDrive/cs234_project'  # Change this to your actual path
sys.path.append(project_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Mathematical Definition of the PDE

The 2D advection equation describes the transport of a scalar quantity $q(x,y,t)$ by a velocity field $(u_x, u_y)$:

$$\frac{\partial q}{\partial t} + u_x \frac{\partial q}{\partial x} + u_y \frac{\partial q}{\partial y} = 0$$

Our specific parameters are:
- $u_x = 0.5$ (x-velocity)
- $u_y = 1.0$ (y-velocity)
- Domain: $\Omega = [0,1] \times [0,1]$
- Boundary conditions: Periodic in both x and y directions
- Initial condition: A square pulse with $q = 1.0$ in $[0.1,0.6] \times [0.1,0.6]$ and $q = 0.1$ elsewhere
- Grid: $50 \times 50$ points
- Time domain: $t \in [0, 2.0]$
- Number of time steps: 20

## 2. Solution Representation

### 3.1 Numerical Solution

After solving the PDE, we obtain the solution at each grid point and time step. We organize this solution data into a matrix $Y$ with the following structure:

$$Y =
\begin{bmatrix}
y(x_1, y_1, t_1) & y(x_1, y_1, t_2) & \cdots & y(x_1, y_1, t_m) \\
y(x_2, y_1, t_1) & y(x_2, y_1, t_2) & \cdots & y(x_2, y_1, t_m) \\
\vdots & \vdots & \ddots & \vdots \\
y(x_n, y_p, t_1) & y(x_n, y_p, t_2) & \cdots & y(x_n, y_p, t_m)
\end{bmatrix}$$

- Each row corresponds to a specific spatial location $(x_i, y_j)$
- Each column corresponds to a time step $t_k$
- The matrix has dimensions $(n \cdot p) \times m$ where:
  - $n = 50$ (number of x grid points)
  - $p = 50$ (number of y grid points)
  - $m = 21$ (number of time steps, including initial condition)


In [4]:
from pde.AdvectionEquation import Advection2D, Adv2dModelConfig
config = Adv2dModelConfig()
adv_obj = Advection2D(config)
# Generate initial condition
init_cond = adv_obj.initial_condition()
# Solve PDE
results = adv_obj.step(init_cond)
# Get actual dimensions
n = adv_obj.nx  # number of x grid points
p = adv_obj.ny  # number of y grid points
actual_frames = len(results)  # actual number of frames returned
print(f"Grid dimensions: {n}x{p}")
print(f"Number of frames returned by solver: {actual_frames}")
# Total time steps including initial condition
m = actual_frames + 1
# Create the Y matrix with shape (n*p, m)
Y = np.zeros((n * p, m))
# Fill in the initial condition (t=0)
Y[:, 0] = init_cond.reshape(-1)
# Fill in the remaining time steps
for t in range(1, m):
    # Get the solution at time step t-1 (frames are 0-indexed)
    q_t = results[t-1].q[0]
        # Flatten the 2D spatial grid into a column
    Y[:, t] = q_t.reshape(-1)

print(f"\nY matrix shape: {Y.shape}")
# Verify a few values from the Y matrix
print(f"\nValue at (x=10, y=10, t=0): {Y[10 + 10*n, 0]}")
middle_t = m // 2
print(f"Value at (x=25, y=25, t={middle_t}): {Y[25 + 25*n, middle_t]}")
print(Y)

Grid dimensions: 50x50
Number of frames returned by solver: 21

Y matrix shape: (2500, 22)

Value at (x=10, y=10, t=0): 1.0
Value at (x=25, y=25, t=11): 0.10348274294673412
[[0.1        0.1        0.1        ... 0.64481541 0.17853983 0.10000994]
 [0.1        0.1        0.1        ... 0.64516202 0.20211766 0.10008263]
 [0.1        0.1        0.1        ... 0.64522344 0.21437734 0.10047498]
 ...
 [0.1        0.1        0.1        ... 0.42344887 0.10203262 0.1       ]
 [0.1        0.1        0.1        ... 0.43671546 0.10777547 0.10000002]
 [0.1        0.1        0.1        ... 0.44079976 0.11904474 0.10000029]]


The covariance function  is defined in Equation (20) of the paper as:



# Covariance Function and Discretization

## Continuous Covariance Function R(x, ζ)

The covariance function R(x, ζ) is defined in Equation (20) of the paper as:

$$\int_{\Omega} R(x, \zeta) \phi_i(\zeta) d\zeta = \lambda_i \phi_i(x)$$

Where:
- R(x, ζ) = ⟨y(x, t)y(ζ, t)⟩ is the spatial two-point correlation function
- Measures how strongly the values of y(x,t) and y(ζ,t) are correlated over time
- φ_i(x) are the eigenfunctions of R(x, ζ), obtained from the Karhunen–Loève Decomposition (KLD)

## Discretization of Covariance Matrix

From Equation (24) of the paper, the temporal correlation is defined as:

$$C_{tk} = \frac{1}{l} \int_{\Omega} y(\zeta, t) y(\zeta, k) d\zeta$$

Where:
- C_{tk} represents the temporal correlation between states at different time steps
- The integral over Ω converts the continuous form into a discrete covariance matrix

### Discrete Covariance Matrix Formulation

The basic discrete form:

$$R_{ij} = \frac{1}{m} \sum_{t=1}^{m} y(x_i, t) y(x_j, t)$$

### Mean-Subtracted Covariance Matrix

To center the data, use mean subtraction:

$$R_{ij} = \frac{1}{m} \sum_{t=1}^{m} \left( y(x_i, t) - \bar{y}(x_i) \right) \left( y(x_j, t) - \bar{y}(x_j) \right)$$



## Source
Appendix A, Equations (20) and (24) in the referenced paper.


In [5]:
from reward.reward import RewardCalculator
# Reshape Y from (n*p, m) to (n, p, m) for RewardCalculator
Y_3d = Y.reshape(n, p, m)
# Create RewardCalculator instance
reward_calc = RewardCalculator(Y_3d)
# Compute covariance matrix
cov_matrix = reward_calc.compute_covariance_matrix()
print(f"Covariance matrix shape: {cov_matrix.shape}")
print(f"Expected shape: ({n*p}, {n*p})")
type(cov_matrix)


Covariance matrix shape: (2500, 2500)
Expected shape: (2500, 2500)


numpy.ndarray

## **3. Solve the Eigenvalue Problem**

decompose the covariance matrix to find dominant spatial patterns using:

$$
R \Phi = \Lambda \Phi
$$

where:
- $\Phi$ contains **eigenvectors** (KLD modes).
- $\Lambda$ is a **diagonal matrix of eigenvalues** (variance captured by each mode).
- Eigenvalues are sorted in **descending order**, ensuring the most important modes are selected.

**Reference:** Appendix A, Equation (20).

---




In [7]:
# 3. Eigenvalue Decomposition
# Now that we've fixed the RewardCalculator class, we can use its methods directly

# Solve eigenvalue problem
eigenvalues, eigenvectors = reward_calc.solve_eigenvalue_problem()
print(f"Number of eigenvalues: {len(eigenvalues)}")
print(f"Top 5 eigenvalues: {eigenvalues[:5]}")



lol
Number of eigenvalues: 2500
Top 5 eigenvalues: [87.61859766 67.42263347 53.24866286 47.85788616 26.26264473]


## **3. Select KLD Modes**

The total **energy captured** by the first k modes is given by:

$$
E_k = \frac{\sum_{i=1}^{k} \lambda_i}{\sum_{j=1}^{n} \lambda_j}
$$

We choose k such that:

$$
E_k \geq 0.99
$$

This ensures we retain **99% of the system's variance**.

**Reference:** Appendix A, Equation (25)

In [8]:
# 4. Mode Selection
# Calculate cumulative energy
cumulative_energy = np.cumsum(eigenvalues) / np.sum(eigenvalues)
threshold = 0.99
num_modes = np.searchsorted(cumulative_energy, threshold) + 1
print(f"Number of modes required to capture {threshold*100}% of energy: {num_modes}")
# Select KLD modes
selected_modes = reward_calc.select_KLD_modes(num_modes)
print(f"Selected modes shape: {selected_modes.shape}")



Number of modes required to capture 99.0% of energy: 13
Selected modes shape: (2500, 13)



## **4. Compute the Reward Function**

The **reward function** is based on the **information gain** from sensor placement.

1. **Extract the sensor locations** from the selected KLD modes:

$$
P_m = \text{selected\_modes}[ \text{sensor\_indices}, :]
$$

2. **Compute the reduced observability matrix**:

$$
T_m = P_m^T P_m
$$

3. **Define the reward as the determinant**:

$$
\text{Reward} = \log \det(T_m)
$$

Equation(12)

In [10]:

# 5. Reward Calculation
# Calculate reward for a sample set of sensor positions
example_sensors = [(25, 25), (10, 40), (30, 15)]
flat_indices = [i + j * n for i, j in example_sensors]
reward = reward_calc.compute_reward_function(flat_indices)
print(f"Reward for example sensor configuration: {reward}")




Reward for example sensor configuration: 5.589776985776396e-08


In [ ]:
""""
# 6. Find Optimal Single Sensor Location
print("\nFinding optimal single sensor location...")
best_reward = -float('inf')
best_location = None

# Sample grid points to reduce computation (every 5th point)
step = 5
grid_samples = [(i, j) for i in range(0, n, step) for j in range(0, p, step)]
print(f"Testing {len(grid_samples)} locations...")

for i, j in grid_samples:
    flat_idx = i + j * n
    current_reward = reward_calc.compute_reward_function([flat_idx])

    if current_reward > best_reward:
        best_reward = current_reward
        best_location = (i, j)

print(f"Best single sensor location: {best_location}")
print(f"Reward value: {best_reward}")"""